# REMOVING DUPLICATES

In [13]:
import pandas as pd
data = {"Name": ["James", "Alice", "Phil", "James"],
		"Age": [24, 28, 40, 24],
		"Sex": ["Male", "Female", "Male", "Male"],
       "Occupation":['Town Mayor','Housewife','Fisherman','Town Mayor']}
dataset = pd.DataFrame(data)
dataset

,Age,Name,Occupation,Sex
0,24,James,Town Mayor,Male
1,28,Alice,Housewife,Female
2,40,Phil,Fisherman,Male
3,24,James,Town Mayor,Male


## DROP DUPLICATE ROWS
drop_duplicates returns only the dataframe’s unique values. The drop_duplicates() returns only the dataframe’s unique values. In the sample below, James is duplicate where all records on each cloumn are the same

In [14]:
df = dataset.drop_duplicates()
print(df)
print()
print(dataset)

   Age   Name  Occupation     Sex
0   24  James  Town Mayor    Male
1   28  Alice   Housewife  Female
2   40   Phil   Fisherman    Male

   Age   Name  Occupation     Sex
0   24  James  Town Mayor    Male
1   28  Alice   Housewife  Female
2   40   Phil   Fisherman    Male
3   24  James  Town Mayor    Male


## REMOVE DUPLICATE BASE ON SUBSET (COLUMN)
To remove duplicates of only one or a subset of columns, specify subset as the individual column or list of columns that should be unique. To do this conditional on a different column’s value, you can sort_values(colname) and specify keep equals either first or last.

In [17]:
import pandas as pd
data = {"Name": ["James", "Alice", "Phil", "James"],
		"Age": [24, 28, 40, 25],
		"Sex": ["Male", "Female", "Male", "Male"],
       "Occupation":['Town Mayor','Housewife','Fisherman','Town Mayor']}
dataset = pd.DataFrame(data)
dataset

,Age,Name,Occupation,Sex
0,24,James,Town Mayor,Male
1,28,Alice,Housewife,Female
2,40,Phil,Fisherman,Male
3,25,James,Town Mayor,Male


If we use the drop_duplicates() function, James will still have two entries since the age is still different on each record. 

In [19]:
df = dataset.sort_values('Age', ascending=False)
df = df.drop_duplicates(subset='Name', keep='first')
print(df)

   Age   Name  Occupation     Sex
2   40   Phil   Fisherman    Male
1   28  Alice   Housewife  Female
3   25  James  Town Mayor    Male


## FINDING DUPLICATES
Python for Data Analysis, defines Data Preparation as “cleaning, munging, combining, normalizing, reshaping, slicing, dicing, and transforming data for analysis.”

In [45]:
import pandas as pd
dataset = pd.read_csv('Order_20190115.csv')
dataset.head()

,order_date,order_id,customer_name,order_item_cd,order_item_qty,order_item_price
0,2019-01-01,342455670,Joel P. Laurel,IC53344,1,2.09
1,2019-01-01,342455670,Joel P. Laurel,I2T3344,3,12.22
2,2019-01-01,342455670,Joel P. Laurel,I2T3344,3,12.22
3,2019-01-01,342455670,Luis T. Quezon,IM5J601,1,12.22
4,2019-01-02,352455677,Kaleb S. Pablo,I2T31124,3,6.29


### CREATE NEW BOLEAN COLUMN

In [46]:
dataset['is_duplicated'] = dataset.duplicated(['order_id', 'order_item_cd'])
dataset.head()

,order_date,order_id,customer_name,order_item_cd,order_item_qty,order_item_price,is_duplicated
0,2019-01-01,342455670,Joel P. Laurel,IC53344,1,2.09,False
1,2019-01-01,342455670,Joel P. Laurel,I2T3344,3,12.22,False
2,2019-01-01,342455670,Joel P. Laurel,I2T3344,3,12.22,True
3,2019-01-01,342455670,Luis T. Quezon,IM5J601,1,12.22,False
4,2019-01-02,352455677,Kaleb S. Pablo,I2T31124,3,6.29,False


### CHECK THE MAGNITUDE 
Check how big of an issue for the duplictes. Sum up all of the rows that were marked as a duplicate. This will give the number of duplicate line items in the dataset.

In [47]:
dataset['is_duplicated'].sum()

2

### FILTER THE RECORDS 

In [48]:
dataset[dataset.is_duplicated==True]

,order_date,order_id,customer_name,order_item_cd,order_item_qty,order_item_price,is_duplicated
2,2019-01-01,342455670,Joel P. Laurel,I2T3344,3,12.22,True
8,2019-01-05,382457651,Sunji A. Que,IG5H601,1,14.67,True


### QUANTIFY THE DUPLICATE RECORDS 
Quantify the impact of duplicate line items in terms of over counted units sold

In [63]:
dataset[dataset['is_duplicated']].order_item_qty.sum()

4

In [64]:
dataset['line_item_total'] = dataset.order_item_qty * dataset.order_item_price
dataset[dataset['is_duplicated']].line_item_total.sum()

51.330000000000005

### SAVE THE GOOD

In [65]:
df_nodup = dataset.loc[dataset['is_duplicated'] == False]
df_nodup

,order_date,order_id,customer_name,order_item_cd,order_item_qty,order_item_price,is_duplicated,line_item_total
0,2019-01-01,342455670,Joel P. Laurel,IC53344,1,2.09,False,2.09
1,2019-01-01,342455670,Joel P. Laurel,I2T3344,3,12.22,False,36.66
3,2019-01-01,342455670,Luis T. Quezon,IM5J601,1,12.22,False,12.22
4,2019-01-02,352455677,Kaleb S. Pablo,I2T31124,3,6.29,False,18.87
5,2019-01-03,353325634,Joel P. Mateo,IQ52601,1,13.57,False,13.57
6,2019-01-04,367125512,Poji A. Sun,I2T3314,3,13.78,False,41.34
7,2019-01-05,382457651,Sunji A. Que,IG5H601,1,14.67,False,14.67


### SANITY CHECK

In [66]:
df_nodup[df_nodup['is_duplicated']].line_item_total.sum()

0.0

In [67]:
df_nodup.to_csv('2019sales_nodup.csv', encoding='utf-8')